In [1]:
import pandas as pd
import time as time
import random
import os
from urllib.parse import urlparse, urljoin
from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from azure.storage.blob import BlobServiceClient
from config import new_chromedriver_path, user_agents, path, connection_string
from web_scraper import WebScraper
from vpn import VPNChanger

In [2]:
scraper = WebScraper(new_chromedriver_path, user_agents, path)

In [3]:
driver = scraper.header_configuration(headless=False)
driver.get(f'https://lista.mercadolivre.com.br/')

In [4]:
# Encontrar o elemento de entrada e inserir um texto de pesquisa
search_box = driver.find_element_by_id('nav-search-input')
search_box.clear()
search_box.send_keys('iphone')

# Encontrar o botão de envio e clicar nele
submit_button = driver.find_element_by_css_selector('button[type="submit"]')
submit_button.click()

C:\Users\victo\AppData\Local\Temp\ipykernel_26392\757819596.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search_box = driver.find_element_by_id('nav-search-input')
C:\Users\victo\AppData\Local\Temp\ipykernel_26392\757819596.py:7: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  submit_button = driver.find_element_by_css_selector('button[type="submit"]')


In [5]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [28]:
li = soup.find_all('li', class_='ui-search-layout__item')

In [45]:
informations = []

for item in li:
    title = item.find('h2', class_='ui-search-item__title').text
    price = item.find_all('span', class_='andes-money-amount__fraction')
    original_price = item.find('s', class_='andes-money-amount ui-search-price__part ui-search-price__part--small ui-search-price__original-value andes-money-amount--previous andes-money-amount--cents-superscript andes-money-amount--compact')
    real_price = item.find('div', class_='ui-search-price__second-line').find('span', class_='andes-money-amount__fraction').text
    search_link = item.find('a', class_='ui-search-link').get('href')
    photo_link = item.find('img', class_='ui-search-result-image__element').get('src')

    structure_dict = {
        'title': title,
        'original_price': original_price.text if original_price else None,
        'real_price': real_price,
        'search_link': search_link,
        'photo_link': photo_link
    }

    informations.append(structure_dict)

df_scrap = pd.DataFrame(informations)

,title,original_price,real_price,search_link,photo_link
0,Apple iPhone 15 Pro (1 TB) - Titânio Branco,R$14.099,13.099,https://click1.mercadolivre.com.br/mclics/clic...,https://http2.mlstatic.com/D_NQ_NP_812116-MLA7...
1,Apple iPhone 13 (128 GB) - Estelar - Distribui...,R$7.599,4.206,https://www.mercadolivre.com.br/apple-iphone-1...,https://http2.mlstatic.com/D_NQ_NP_916682-MLA4...
2,Apple iPhone 11 (64 GB) Preto,None,2.124,https://www.mercadolivre.com.br/apple-iphone-1...,https://http2.mlstatic.com/D_NQ_NP_832687-MLA4...
3,iPhone 13 De 128 GB Cor Meia Noite Apple - Dis...,R$7.599,4.206,https://www.mercadolivre.com.br/iphone-13-de-1...,https://http2.mlstatic.com/D_NQ_NP_790522-MLA4...
4,Apple iPhone 13 (128 GB) - Azul - Distribuidor...,R$7.599,4.206,https://www.mercadolivre.com.br/apple-iphone-1...,https://http2.mlstatic.com/D_NQ_NP_640758-MLA4...
5,Apple iPhone 14 (128 GB) - Meia-noite - Distri...,R$6.999,4.779,https://www.mercadolivre.com.br/apple-iphone-1...,https://http2.mlstatic.com/D_NQ_NP_881016-MLM5...
6,Apple iPhone 14 (128 GB) - Estelar - Distribui...,R$6.999,4.779,https://www.mercadolivre.com.br/apple-iphone-1...,https://http2.mlstatic.com/D_NQ_NP_918579-MLM5...
7,Apple iPhone 15 Pro Max (256 GB) - Titânio Pre...,None,10.999,https://www.mercadolivre.com.br/apple-iphone-1...,https://http2.mlstatic.com/D_NQ_NP_768125-MLA7...
8,Apple iPhone 14 (128 GB) - Azul - Distribuidor...,R$8.999,4.779,https://www.mercadolivre.com.br/apple-iphone-1...,https://http2.mlstatic.com/D_NQ_NP_864844-MLM5...
9,Apple iPhone 15 Pro Max (256 GB) - Titânio Azu...,None,10.999,https://www.mercadolivre.com.br/apple-iphone-1...,https://http2.mlstatic.com/D_NQ_NP_912227-MLA7...


In [47]:
soup.find('li', class_='andes-pagination__button andes-pagination__button--next').a

<li class="andes-pagination__button andes-pagination__button--next"><a class="andes-pagination__link ui-search-link" href="https://lista.mercadolivre.com.br/celulares-telefones/celulares-smartphones/iphone_Desde_51_NoIndex_True" rel="nofollow" role="button" title="Seguinte"><span class="andes-pagination__arrow-title">Seguinte</span><span class="andes-pagination__arrow"></span></a></li>

In [48]:
wait = WebDriverWait(driver, 10)
next_button = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'li.andes-pagination__button--next a')))

next_button.click()